In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
data=[]

for dir, x, files in os.walk('flower_photos'):
    label=dir.split('/')[-1]

    for file in files:
        path=os.path.join(dir, file)
        data.append([path, label])

df=pd.DataFrame(data, columns=["file_path", "label"])
print(df)

                                             file_path   label
0       flower_photos/roses/16209331331_343c899d38.jpg   roses
1        flower_photos/roses/5777669976_a205f61e5b.jpg   roses
2      flower_photos/roses/4860145119_b1c3cbaa4e_n.jpg   roses
3       flower_photos/roses/15011625580_7974c44bce.jpg   roses
4     flower_photos/roses/17953368844_be3d18cf30_m.jpg   roses
...                                                ...     ...
3665     flower_photos/tulips/134143359_71fa8dd9a4.jpg  tulips
3666    flower_photos/tulips/3637371174_a8dfcc1b35.jpg  tulips
3667  flower_photos/tulips/6948239566_0ac0a124ee_n.jpg  tulips
3668    flower_photos/tulips/2834890466_1cf220fba1.jpg  tulips
3669   flower_photos/tulips/13953090784_0c7d7a904e.jpg  tulips

[3670 rows x 2 columns]


In [3]:
datagen=ImageDataGenerator(
    validation_split=0.2
)

In [4]:
train_gen= datagen.flow_from_dataframe(
    dataframe=df,
    x_col='file_path',
    y_col='label',
    target_siz=(224, 224),
    color_mode='rgb',
    bach_size= 32,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

Found 2936 validated image filenames belonging to 5 classes.


In [5]:
valid_gen= datagen.flow_from_dataframe(
    dataframe=df,
    x_col='file_path',
    y_col='label',
    target_siz=(224, 224),
    color_mode='rgb',
    bach_size= 32,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

Found 734 validated image filenames belonging to 5 classes.


In [6]:
base_model = EfficientNetB3(
    include_top= False, 
    weights='imagenet',
    input_shape=(224, 224, 3), 
    pooling='max'
)

model = tf.keras.models.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(rate=.45, seed=123),
    layers.Dense(5, activation='softmax')
])


model.compile(
    Adamax(learning_rate= .0001), 
    loss = 'categorical_crossentropy',
    metrics=[
        'acc', 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall(), 
        tf.keras.metrics.AUC()
    ]
)


In [7]:
early_stopper= EarlyStopping(
    patience=10,
    min_delta=0,
    monitor='val_loss',
    verbose=0,
    restore_best_weights=True,
    baseline=None
)

In [8]:
model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=5,
    validation_steps=None,
    shuffle=False,
    callbacks=early_stopper,
    verbose=1
)

Epoch 1/5


/Users/Cohort23/Library/Python/3.10/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "efficientnetb3" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, 256, 256, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 256, 256, 3), dtype=float32)
  • training=True
  • mask=None